In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 60 kB/s 
     |████████████████████████████████| 198 kB 71.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=f308fd933b1e90c371b7dfb6e188a2348f30864188ff6925afcf8898e7fd532f
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [4]:
from pyspark import SparkContext, SparkConf

In [5]:
conf = SparkConf().setAppName('Spark Prac').setMaster('local')
sc = SparkContext(conf = conf)

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Admission_Prediction.csv to Admission_Prediction.csv


In [7]:
import io
data1 = io.BytesIO(uploaded['Admission_Prediction.csv'])    

In [8]:
data = sc.textFile('Admission_Prediction.csv')

#### Import PySpark ML Libraries

In [9]:
from __future__ import print_function
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [10]:
spark = SparkSession.builder.appName("Spark ML Algo").getOrCreate()

In [11]:
dataframe = spark.read.csv("Admission_Prediction.csv", header=True)

In [12]:
type(dataframe)

pyspark.sql.dataframe.DataFrame

In [13]:
dataframe.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [14]:
dataframe.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [15]:
from pyspark.sql.functions import col
new_dataframe = dataframe.select(*(col(c).cast("float").alias(c) for c in dataframe.columns))

In [16]:
new_dataframe.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [17]:
from pyspark.sql.functions import col, count, isnan, when

In [18]:
for c in new_dataframe.columns:
  print(c)

GRE Score
TOEFL Score
University Rating
SOP
LOR
CGPA
Research
Chance of Admit


In [19]:
### Checking for null ir nan type values in out columns
new_dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataframe.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [20]:
from pyspark.ml.feature import Imputer

In [21]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_dataframe)

imputed_data = model.transform(new_dataframe)

In [22]:
imputed_data

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float, Chance of Admit: float]

In [23]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [24]:
features = imputed_data.drop('Chance of Admit')

In [25]:
features

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float]

In [26]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler( inputCols=features.columns,outputCol="features")

In [27]:
output = assembler.transform(imputed_data)

#### Linear Regressor

In [28]:
output= output.select("features", "Chance of Admit")

In [29]:
output = assembler.transform(imputed_data)

In [30]:
output= output.select("features", "Chance of Admit")

In [31]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [32]:
train_df.show()
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,4...|           0.34|
|[298.0,99.0,1.0,1...|           0.53|
|[298.0,105.0,3.0,...|           0.69|
+--------------------+---------------+
only showing top 20 rows

+--------------------+---------------+

In [33]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit')
linear_model = lin_reg.fit(train_df)

In [34]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [0.001164748062041676,0.002731724659456793,0.002444087344080545,0.010395272333342713,0.013826716932124027,0.12110420909124693,0.035173941251165235]
Intercept: -1.092643840771405


In [35]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.059702
r2: 0.814640


In [36]:
# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit","features").show()

+-------------------+---------------+--------------------+
|         prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
| 0.5743770157191435|           0.64|[293.0,97.0,2.0,2...|
|0.42784557909356846|           0.46|[295.0,93.0,1.0,2...|
| 0.5098462486231841|           0.49|[296.0,97.0,2.0,1...|
| 0.5226605254433108|           0.43|[297.0,96.0,2.0,2...|
| 0.4738659431076402|           0.34|[297.0,96.0,2.0,2...|
| 0.5222211495690103|           0.59|[297.0,98.0,2.0,2...|
| 0.5206634832977826|           0.44|[298.0,98.0,2.0,1...|
| 0.6142032386845719|           0.58|[298.0,100.0,3.0,...|
| 0.5303688890009308|           0.54|[298.0,101.0,2.0,...|
| 0.5948053448188801|           0.53|[298.0,101.0,4.0,...|
| 0.5417731818487697|           0.46|[298.0,107.187751...|
| 0.5559534778676702|           0.38|[299.0,97.0,3.0,5...|
| 0.5364216304421325|           0.51|[299.0,100.0,2.0,...|
| 0.6154300933808228|           0.65|[300.0,97.0,2.0,3..

In [37]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit",metricName="r2")
print("R Squared (R2) on test data =", pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.822919129851789


#### Random Forest Regressor

In [38]:
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

In [39]:
featureIndexer = featureIndexer.transform(output)

In [40]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit")

In [41]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

In [42]:
training.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,99.0,2.0,3...|           0.47|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,98.0,2.0,4...|           0.34|
|[298.0,99.0,1.0,1...|           0.53|
|[298.0,100.0,3.0,...|           0.58|
|[298.0,101.0,2.0,...|           0.54|
|[298.0,101.0,4.0,...|           0.53|
+--------------------+---------------+
only showing top 20 rows



In [43]:
test.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[294.0,93.0,1.0,1...|           0.46|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,99.0,2.0,2...|           0.57|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,100.0,1.0,...|           0.52|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,105.0,3.0,...|           0.69|
|[298.0,107.187751...|           0.46|
|[299.0,94.0,1.0,1...|           0.42|
|[299.0,97.0,3.0,5...|           0.38|
|[300.0,97.0,2.0,3...|           0.65|
|[300.0,98.0,1.0,2...|           0.61|
|[300.0,105.0,1.0,...|           0.58|
|[301.0,96.0,1.0,3...|           0.54|
|[301.0,98.0,1.0,2...|           0.67|
|[301.0,100.0,3.0,...|           0.67|
|[301.0,102.0,3.0,...|           0.68|
+--------------------+---------------+
only showing top 20 rows



In [44]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit" )

In [50]:
# Train model.  This also runs the indexer.
model = random_forest_reg.fit(test)

In [51]:
# Make predictions.
predictions = model.transform(test)

In [52]:
predictions.show()

+--------------------+---------------+-------------------+
|     indexedFeatures|Chance of Admit|         prediction|
+--------------------+---------------+-------------------+
|[290.0,100.0,1.0,...|           0.47| 0.4777091139306625|
|[294.0,93.0,1.0,1...|           0.46|0.46320655139784017|
|[295.0,93.0,1.0,2...|           0.46| 0.4712184565549805|
|[295.0,99.0,2.0,2...|           0.57| 0.5531493390883001|
|[296.0,97.0,2.0,1...|           0.49| 0.5342833346057506|
|[296.0,101.0,1.0,...|            0.6| 0.5833469651425871|
|[297.0,100.0,1.0,...|           0.52| 0.5613966458523401|
|[298.0,97.0,3.121...|           0.45| 0.4776309536681289|
|[298.0,98.0,2.0,1...|           0.44| 0.4717858990778526|
|[298.0,105.0,3.0,...|           0.69| 0.6983823208545262|
|[298.0,107.187751...|           0.46| 0.5209569958580408|
|[299.0,94.0,1.0,1...|           0.42| 0.4394690415014823|
|[299.0,97.0,3.0,5...|           0.38|0.49078205158440474|
|[300.0,97.0,2.0,3...|           0.65| 0.603419511228749

In [53]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data =  0.04024199137120952


In [54]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit", predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.9153660775567585
